# importing data

In [3]:
"""
Simple tester for the deep3d
"""

import tensorflow as tf
import Deep3D as deep3d
import utils
import numpy as np
import os
import os.path
import h5py
import matplotlib as plt

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
inria_file = '/a/data/deep3d/inria_data.h5'
h5f = h5py.File(inria_file,'r')

X_train = h5f['X_0'][:400,10:170,16:304,:]
Y_train = h5f['Y_0'][:400,10:170,16:304,:]
X_val = h5f['X_0'][400:,10:170,16:304,:]
Y_val = h5f['Y_0'][400:,10:170,16:304,:]
   
    
h5f.close()

In [5]:
print Y_val.shape

(100, 160, 288, 3)


In [6]:
num_batches = 10
batchsize = 40
device1 = '/gpu:0'
#device1 = '/cpu:0'
with tf.device(device1):
    sess = tf.Session()

    images = tf.placeholder(tf.float32, [batchsize, 160, 288, 3])
    true_out = tf.placeholder(tf.float32, [batchsize, 160, 288, 3])
    
    train_mode = tf.placeholder(tf.bool)

    net = deep3d.Deep3Dnet('./vgg19.npy')
    net.build(images, train_mode)

    # print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print 'var count:'
    print(net.get_var_count())

    sess.run(tf.global_variables_initializer())

    # simple 1-step training
    cost = tf.reduce_sum((net.prob - true_out) ** 2)
    train = tf.train.GradientDescentOptimizer(0.002).minimize(cost)
    for i in xrange(num_batches):
        image_mask = np.random.choice(X_train.shape[0],batchsize)
        images_in = X_train[image_mask,:,:,:]
        labels_in = Y_train[image_mask,:,:,:]
        sess.run(train, feed_dict={images: images_in, true_out: labels_in, train_mode: True})
        print 'cost:'
        print cost.eval(session=sess)
    # test classification again, should have a higher probability about tiger
    #prob = sess.run(vgg.prob, feed_dict={images: batch1, train_mode: False})
    #utils.print_prob(prob[0], './synset.txt')

    # test save
    #vgg.save_npy(sess, './test-save.npy')

var count:
42
138408071


ResourceExhaustedError: OOM when allocating tensor with shape[40,160,288,33]
	 [[Node: gradients/Slice_11_grad/Pad = Pad[T=DT_FLOAT, Tpaddings=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](gradients/Mul_5_grad/tuple/control_dependency, gradients/Slice_11_grad/concat)]]

Caused by op u'gradients/Slice_11_grad/Pad', defined at:
  File "/usr/lib64/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib64/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/a/h/jhohra01/Envs/deep-venv/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/a/h/jhohra01/Envs/deep-venv/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/a/h/jhohra01/Envs/deep-venv/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 405, in start
    ioloop.IOLoop.instance().start()
  File "/a/h/jhohra01/Env

# testing the network

In [ ]:
img1 = utils.load_image("./test_data/tiger.jpeg")
img1_true_result = [1 if i == 292 else 0 for i in range(1000)]  # 1-hot result for tiger

batch1 = img1.reshape((1, 180, 320, 3))

with tf.device('/gpu:0'):
    sess = tf.Session()

    images = tf.placeholder(tf.float32, [1, 180, 320, 3])
    true_out = tf.placeholder(tf.float32, [1, 1000])
    train_mode = tf.placeholder(tf.bool)

    vgg = deep3d.Deep3Dnet('./deep3d-save.npy')
    vgg.build(images, train_mode)

    # print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print(vgg.get_var_count())

    sess.run(tf.global_variables_initializer())

    # test classification
    prob = sess.run(vgg.prob, feed_dict={images: batch1, train_mode: False})
    utils.print_prob(prob[0], './synset.txt')

    # simple 1-step training
    cost = tf.reduce_sum((vgg.prob - true_out) ** 2)
    train = tf.train.GradientDescentOptimizer(0.0001).minimize(cost)
    sess.run(train, feed_dict={images: batch1, true_out: [img1_true_result], train_mode: True})

    # test classification again, should have a higher probability about tiger
    prob = sess.run(vgg.prob, feed_dict={images: batch1, train_mode: False})
    utils.print_prob(prob[0], './synset.txt')

    # test save
    vgg.save_npy(sess, './test-save.npy')